# Code Generation and Evaluation

In [1]:
from human_eval.data import read_problems
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import random, time, datetime
from datasets import load_dataset
from mistralai import Mistral
import json

%load_ext autoreload
%autoreload 2

In [2]:
from prompts import ( complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,
                      complete_task_prompt_basic, complete_task_prompt, complete_task_prompt_full,
                      reflection_prompt_basic, reflection_prompt, reflection_prompt_full, )
from helpers import get_tokenizer, get_model, clean_code, clean_code_light, is_function, write_jsonl

In [3]:
# select model, propmt, temperature, top_p
models = [
    ('ministral-3b-latest', 'mistral_3b'),
    ('ministral-8b-latest', 'mistral_8B'),
    ('open-mistral-nemo', 'nemo'),
    #('open-codestral-mamba', 'codestral_mamba'),
    #('open-mistral-7b', 'mistral_7b'),    
]
# actual prompt & prompt name for log file name
prompts_and_names = [
    (complete_code_prompt_basic, 'complete_code_prompt_basic'),            
    (complete_code_prompt,       'complete_code_prompt'),
    (complete_code_prompt_full,  'complete_code_prompt_full'),
    (complete_task_prompt_basic, 'complete_task_prompt_basic'),
    (complete_task_prompt,       'complete_task_prompt'),
    (complete_task_prompt_full,  'complete_task_prompt_full'), ]
# reflection prompts & their names for log file name
reflection_prompts_and_names = [
    ( reflection_prompt_basic, 'reflection_prompt_basic' ),
    ( reflection_prompt, 'reflection_prompt' ),
    ( reflection_prompt_full, 'reflection_prompt_full' ), ]
# temperature and top_k values and labels for file names
temperature_values = [
    (1.0,  'temperature1.0'),
    (0.75, 'temperature0.75'),
    (0.5,  'temperature0.5'), ]
top_p_values = [
    (1.0,  'topP1.0'),
    (0.75, 'topP0.75'),
    (0.5,  'topP0.5'), ]

model_idx            = 2
prompt_idx           = 0
relection_prompt_idx = 0
temperature_idx      = 0
top_p_idx            = 0

model_name, model_nickname     = models[ model_idx ]
my_prompt, my_prompt_label     = prompts_and_names[ prompt_idx ]
my_reflection_prompt, my_reflection_prompt_label = reflection_prompts_and_names[ relection_prompt_idx ]
TEMPERATURE, temperature_label = temperature_values[ temperature_idx ]
TOP_P, top_p_label             = top_p_values[ top_p_idx ]

SPECIAL_MESSAGE = f'Model temperature: {TEMPERATURE}. Model top_p: {TOP_P}. Model top_k: 50. Model dtype: torch.float32'

print(f'Model name:             {model_name}')
print(f'Model nickname:         {model_nickname}')
print(f'Prompt name:            {my_prompt_label}')
print(f'Relfection prompt name: {my_reflection_prompt_label}')
print(f'\nPrompt:\n{my_prompt.strip().format("Starter Code")}\n')
print(f'Temperature: {TEMPERATURE}. Temperature label: {temperature_label}')
print(f'Top p: {TOP_P}. Top p label: {top_p_label}')
print(f'Special message: {SPECIAL_MESSAGE}')

Model name:             open-mistral-nemo
Model nickname:         nemo
Prompt name:            complete_code_prompt_basic
Relfection prompt name: reflection_prompt_basic

Prompt:
Complete the following Python code:
Starter Code

Temperature: 1.0. Temperature label: temperature1.0
Top p: 1.0. Top p label: topP1.0
Special message: Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32


In [4]:
# save results
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
results_file   = f'logs_local/{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_completions_{time_stamp}.jsonl'
log_file       = f'logs_local/logs/{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_log_{time_stamp}.log'

# log results
for handler in logging.root.handlers[:]:            # overwrite any previous handlers with different formats
    logging.root.removeHandler(handler)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s  %(levelname)s  %(message)s',
    handlers=[
        logging.FileHandler(log_file),              # Log to a file
        logging.StreamHandler(sys.stdout)           # Log to console (default - sys.stderr (red background)
    ]
)

# print job config
logging.info('*****     JOB CONFIGURATION     *****')
logging.info(f'MODEL: {model_name}')
logging.info(f'MODEL NICKNAME: {model_nickname}')
logging.info(SPECIAL_MESSAGE)
logging.info(f'GENERATED CODE SAVED IN: {results_file}')
logging.info(f'THIS LOG FILE: {log_file}')
if my_prompt in [complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,]:
    logging.info(f'PROMPT:\n{my_prompt.lstrip().format("Starter Code")}')
else:
    logging.info(f'\nPROMPT:\n{my_prompt.format("Task Description", "Test Cases")}')
logging.info('*****     END OF JOB CONFIGURATION     *****')

2025-06-11 04:09:55,566  INFO  *****     JOB CONFIGURATION     *****
2025-06-11 04:09:55,567  INFO  MODEL: open-mistral-nemo
2025-06-11 04:09:55,567  INFO  MODEL NICKNAME: nemo
2025-06-11 04:09:55,568  INFO  Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32
2025-06-11 04:09:55,568  INFO  GENERATED CODE SAVED IN: logs_local/nemo_complete_code_prompt_basic_temperature1.0_topP1.0_completions_20250611_040955_5664.jsonl
2025-06-11 04:09:55,568  INFO  THIS LOG FILE: logs_local/logs/nemo_complete_code_prompt_basic_temperature1.0_topP1.0_log_20250611_040955_5664.log
2025-06-11 04:09:55,569  INFO  PROMPT:
Complete the following Python code:
Starter Code

2025-06-11 04:09:55,569  INFO  *****     END OF JOB CONFIGURATION     *****


In [5]:
dataset = load_dataset("openai/openai_humaneval")
counter = 0
for item in dataset["test"]:
    if counter == 1: break
    counter += 1
    for k,v in item.items():
        print(f'{k}:\n{v}\n')
    print('\n', '='*75, '\n', sep='')

task_id:
HumanEval/0

prompt:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


canonical_solution:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


test:


METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False

In [6]:
def api_call(model, prompt):
    ''' One API call to a Mistral model '''
    chat_response = client.chat.complete(
                model=model,
                messages=[{
                        "role": "user",
                        "content": prompt, 
                    }]
                )
    return chat_response.choices[0].message.content

def generate_response_with_backoff(model, prompt, max_retries=10, initial_delay=1, max_delay=16):
    '''
    Make API calls with exponential backoff on errors.
    Args:
        model (str): The model identifier.
        prompt (str): The prompt to send.
        max_retries (int): Maximum number of retries before giving up.
        initial_delay (int or float): Starting delay in seconds.
        max_delay (int or float): Maximum delay between retries.
    Returns:
        str: The chat response content OR f'Error generating a completion:{e}'
        if the max number of attemps has been reached
    '''
    retries = 0
    delay   = initial_delay
    while True:
        try:
            return api_call(model, prompt)
        except Exception as e:
            retries += 1
            if retries > max_retries:
                return f"Error generating a completion after {retries} retries:\n{e}"
            sleep_time = delay + random.uniform(0, 1)    # add jitter
            time.sleep(sleep_time)
            delay = min(delay * 2, max_delay)

In [7]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [8]:
input_prompt = "What is the capital of California?"
print(input_prompt)
print(generate_response_with_backoff(model_name, input_prompt))

What is the capital of California?
2025-06-11 04:10:02,239  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
The capital of California is Sacramento. It has been the state capital since 1854. Sacramento is known for its historic sites, such as the California State Capitol, Old Sacramento, and the Crocker Art Museum. It's also the sixth-largest city in California, with a population of around 500,000 people.


In [9]:
num_samples_per_task = 1
completions  = []

for item in dataset["test"]:
    logging.info(item['task_id'])
    #if item['task_id'] == 'HumanEval/5':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        
        # generate completion
        full_prompt       = my_prompt.lstrip().format( item['prompt'] )
        proposed_solution = generate_response_with_backoff( model_name, full_prompt )
        
        # improve solution
        new_full_prompt   = my_reflection_prompt.lstrip().format(full_prompt, proposed_solution)
        improved_solution = generate_response_with_backoff( model_name, new_full_prompt )
        
        # check if improved solution is a Python function. If not, use the previously proposed solution
        cleaned_improved_solution = clean_code_light(improved_solution).strip()
        if is_function(cleaned_improved_solution):
            final_solution = improved_solution
        else:
            final_solution = proposed_solution
        
        # save and log the results
        completions.append( {'task_id': item['task_id'], 'completion': final_solution} )
        write_jsonl(results_file, completions)
        logging.info('NEW REFLECTION PROMPT:\n' + new_full_prompt + '\n')
        logging.info('IMPROVED COMPLETION:\n' + final_solution + '\n')
        logging.info(f"Time elapsed including backoff: {(time.time() - start_time):.4f} seconds\n")

2025-06-11 04:10:02,298  INFO  HumanEval/0
2025-06-11 04:10:46,581  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-11 04:11:29,589  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-11 04:11:29,599  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following Python code:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.

2025-06-11 04:12:46,810  INFO  IMPROVED COMPLETION:
The proposed solution is correct and efficient. It uses the fact that the integer part of a number can be obtained by using the `int()` function, and the decimal part can be obtained by subtracting the integer part from the original number. Here's the proposed solution:

```python
def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be decomposed into
    an integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.
    >>> truncate_number(3.5)
    0.5
    """
    return number - int(number)
```

Since the proposed solution is already optimal, I will return it as is.

2025-06-11 04:12:46,811  INFO  Time elapsed including backoff: 58.3839 seconds

2025-06-11 04:12:46,812  INFO  HumanEval/3
2025-06-11 04:12:47,798  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 20

2025-06-11 04:15:10,424  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and correct. It handles the case where the input list is empty, and it correctly inserts the delimiter between every two consecutive elements of the input list. Here's a slightly more concise version of the proposed solution:

```python
from typing import List

def intersperse(numbers: List[int], delimeter: int) -> List[int]:
    """ Insert a number 'delimeter' between every two consecutive elements of input list `numbers'
    """
    return [numbers[0]] + [delimeter, num] for num in numbers[1:]]
```

This version of the function uses a list comprehension to achieve the same result as the proposed solution, but with less code. It also returns an empty list if the input list is empty, as required.

The time complexity of this function is O(n), where n is the length of the input list, as it needs to iterate through the list once to insert the delimiter between every two consecutive elements. The 

2025-06-11 04:15:21,640  INFO  IMPROVED COMPLETION:
The proposed solution is almost correct, but it has a small issue. The `eval` function is used to calculate the product, which is not a good practice because it can evaluate any Python expression, which can be a security risk if the input is not trusted. Instead, we can use the `math.prod` function from the `math` module, which is designed to calculate the product of a sequence of numbers.

Here's the optimized version of the proposed solution:

```python
from typing import List, Tuple
import math

def sum_product(numbers: List[int]) -> Tuple[int, int]:
    if not numbers:
        return (0, 1)
    else:
        return (sum(numbers), math.prod(numbers))

print(sum_product([])) # (0, 1)
print(sum_product([1, 2, 3, 4])) # (10, 24)
```

This version of the function is more secure and also more readable. It also handles the case of an empty list correctly, as required by the problem statement.

2025-06-11 04:15:21,641  INFO  Time elapsed 

2025-06-11 04:15:36,795  INFO  IMPROVED COMPLETION:
The proposed solution is correct and efficient for the given requirements. It correctly performs the XOR operation on the binary strings and returns the result as a string. However, there is a minor inconsistency in the docstring example. The expected output for the example '010' XOR '110' should be '100', not '101'.

Here's the optimized version of the proposed solution with the corrected docstring:

```python
from typing import List

def string_xor(a: str, b: str) -> str:
    """
    Input are two strings a and b consisting only of 1s and 0s.
    Perform binary XOR on these inputs and return result also as a string.
    >>> string_xor('010', '110')
    '100'
    """
    result = ""
    for i in range(len(a)):
        if a[i] == b[i]:
            result += '0'
        else:
            result += '1'
    return result
```

This function is already optimal as it has a time complexity of O(n), where n is the length of the input strings.

...

2025-06-11 04:37:41,224  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and correct. It uses a helper function to check if a number is prime, which is an efficient way to solve the problem. The prime-checking function uses the 6k ± 1 optimization, which is a standard and efficient method for checking primality.

Here's the proposed solution again for reference:

```python
def x_or_y(n, x, y):
    def is_prime(n):
        if n <= 1:
            return False
        if n <= 3:
            return True
        if n % 2 == 0 or n % 3 == 0:
            return False
        i = 5
        while i * i <= n:
            if n % i == 0 or n % (i + 2) == 0:
                return False
            i += 6
        return True

    if is_prime(n):
        return x
    else:
        return y
```

Since the proposed solution is already optimal and correct, I will return it as is.

2025-06-11 04:37:41,225  INFO  Time elapsed including backoff: 3.1691 seconds

2025-06-11 04:37:41,226 

2025-06-11 04:38:57,413  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and correct. It correctly calculates the strength of each extension and returns the class name concatenated with the strongest extension name. Here's a brief analysis to confirm its correctness and efficiency:

1. **Correctness**: The solution correctly calculates the strength of each extension by counting the number of uppercase and lowercase letters. It then finds the extension with the maximum strength using the `max` function. If there are multiple extensions with the same strength, it returns the one that comes first in the list, as required.

2. **Efficiency**: The time complexity of the solution is O(n), where n is the number of extensions. This is because the `max` function iterates through the list of extensions once to find the strongest one. The space complexity is O(1), as no additional data structures are used that scale with the input size.

Here's the proposed solution again for 

2025-06-11 04:39:08,610  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and correct. It uses a dictionary to map Roman numerals to their corresponding integer values, then iterates through the dictionary in reverse order to build the Roman numeral string. The while loop ensures that the largest possible Roman numeral is used for each part of the input number.

Here's the proposed solution with added type hints and docstring for better readability:

```python
def int_to_mini_roman(number: int) -> str:
    """
    Given a positive integer, obtain its Roman numeral equivalent as a string,
    and return it in lowercase.

    Args:
        number (int): The positive integer to convert to Roman numeral.

    Returns:
        str: The Roman numeral equivalent of the input number in lowercase.

    Restrictions:
        1 <= num <= 1000

    Examples:
    >>> int_to_mini_roman(19)
    'xix'
    >>> int_to_mini_roman(152)
    'clii'
    >>> int_to_mini_roman(426)
    'cdxx

2025-06-11 04:39:19,118  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and correct. It correctly calculates the total number of carrots eaten and the number of carrots left after the rabbit's meal, taking into account the constraints of the problem. Here's a brief analysis:

1. **Correctness**: The function correctly calculates the total number of carrots eaten by adding the number of carrots already eaten (`number`) to the minimum of the number of carrots needed (`need`) and the number of remaining carrots (`remaining`). This ensures that the rabbit doesn't eat more carrots than are available.

2. **Efficiency**: The function uses two simple mathematical operations (`min` and `max`) and one arithmetic operation (`-`), which makes it very efficient. The time complexity is O(1), meaning it runs in constant time regardless of the input size.

3. **Consistency**: The function consistently returns a list containing the total number of carrots eaten and the number of c

2025-06-11 04:40:44,831  INFO  IMPROVED COMPLETION:
The proposed solution is correct and efficient. It uses the `hashlib` library to compute the MD5 hash of the input string. The `hexdigest()` method is used to convert the hash object to a hexadecimal string.

Here's the proposed solution with some minor improvements:

```python
import hashlib

def string_to_md5(text):
    if not text:
        return None
    return hashlib.md5(text.encode()).hexdigest()

print(string_to_md5('Hello world'))
```

Changes made:

1. Replaced `text == ""` with `not text` for better readability and to handle cases where `text` is `None`.
2. Added a space after the `return` keyword for better code readability.

Since the proposed solution is already optimal, the optimized version is the same as the proposed solution.

2025-06-11 04:40:44,832  INFO  Time elapsed including backoff: 1.9192 seconds

2025-06-11 04:40:44,833  INFO  HumanEval/163
2025-06-11 04:40:45,633  INFO  HTTP Request: POST https://api.mistral

In [10]:
unknown_variable

NameError: name 'unknown_variable' is not defined

# Appendix

In [ ]:
# read a jsonl file
file_name = 'logs/mistral_7b_samples_20241201_005244_0462.jsonl'
data = read_jsonl(file_name)

In [ ]:
data[0]['completion']

In [ ]:
# modify and write back
data2 = []
for i in data:
    new_i = dict()
    new_i['task_id'] = i['task_id']
    new_i['completion'] = add_import(i['completion'])
    data2.append(new_i)

In [ ]:
file_name = 'logs/mistral_7b_samples_20241201_005244_0462_v2.jsonl'
write_jsonl(file_name, data2)